# TFM: PLANIFICACIÓN DE HORARIOS Y AULAS BASADO EN PREFERENCIAS DE PROFESORES Y ALUMNOS

## Universidad Internacional de Valencia

Autor: Ignacio Carrillo Miquel

Director TFM: Cristian Ramírez Atencia

## IMPORTACIÓN DE LIBRERÍAS

En primer lugar se importarán todas las bibliotecas y módulos necesarios para ejecutar el UCTP utilizando algoritmos genéticos. Aquí se importan librerías estándar de Python como `random` y `datetime`, así como bibliotecas de terceros para cálculos numéricos y manipulación de datos (`numpy` y `pandas`). Para la visualización de datos, se emplea `matplotlib.pyplot`. El módulo `os` se utiliza para interactuar con el sistema operativo, permitiendo, por ejemplo, la gestión de archivos.

También se importan clases y funciones específicas del paquete `pymoo`, que es una biblioteca de optimización multiobjetivo. Estas clases y funciones incluyen `Problem` y `Population` para definir el problema y la población inicial, respectivamente. Además, se importan módulos para configurar el algoritmo genético, incluidas las operaciones de muestreo, cruce y mutación. Finalmente, se importan dos algoritmos específicos: el Algoritmo Genético Estándar (`GA`) y el Algoritmo NSGA-II (`NSGA2`), que se emplearán para resolver el problema de optimización multiobjetivo en la planificación de horarios.

In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from datetime import datetime
from collections import Counter
from scipy.stats import wilcoxon

from pymoo.model.problem import Problem
from pymoo.model.population import Population
from pymoo.model.callback import Callback
from pymoo.algorithms.so_genetic_algorithm import GA
from pymoo.algorithms.nsga2 import NSGA2
from pymoo.optimize import minimize
from pymoo.factory import get_sampling
from pymoo.factory import get_crossover
from pymoo.factory import get_mutation
from pymoo.factory import get_selection
from pymoo.factory import get_algorithm
from pymoo.factory import get_termination
from pymoo.factory import get_performance_indicator
from pymoo.operators.crossover.point_crossover import PointCrossover

from pymoo.util.termination.default import MultiObjectiveDefaultTermination
from pymoo.util.termination.default import SingleObjectiveDefaultTermination

## DEFINICIÓN DE DICCIONARIOS Y DATOS INICIALES

En este momento vamos a definir dos diccionarios esenciales para el proyecto. Estos diccionarios servirán como datos de entrada para el algoritmo de optimización que se desarrollará posteriormente, y son cruciales para asegurar que las restricciones del problema, como la disponibilidad de profesores y la capacidad de las aulas, sean debidamente consideradas.

El primer diccionario, `asignaturas`, asocia cada asignatura con tres elementos: el profesor que la imparte, el número de horas semanales que requiere y el número de alumnos matriculados en la asignatura. Cada clave del diccionario representa un identificador único para una asignatura, y el valor asociado es una tupla que contiene la información relevante de la asignatura. Por ejemplo, la asignatura identificada como 'ASIG_034_C4S1_D' es impartida por 'PROFESOR_14', requiere 4 horas a la semana y tiene 30 alumnos matriculados.

El segundo diccionario, `room_capacities`, mapea las capacidades de las aulas disponibles. Las claves son identificadores numéricos para las aulas, y los valores representan la capacidad máxima de cada aula. Por ejemplo, el aula identificada como 0 tiene una capacidad máxima de 25 estudiantes.

In [ ]:
# Diccionario que asocia cada asignatura con su profesor, horas/semana y numero de alumnos matriculados en la asignatura
asignaturas = {
        'ASIG_034_C4S1_D' : ('PROFESOR_14', 4, 30),
        'ASIG_024_C3S1_B' : ('PROFESOR_14', 5, 15),
        'ASIG_035_C1S2_G' : ('PROFESOR_14', 4, 20),
        'ASIG_036_C1S2_H' : ('PROFESOR_14', 1, 20),
        'ASIG_035_C1S2_H' : ('PROFESOR_14', 4, 20),
        
        'ASIG_035_C2S1_E' : ('PROFESOR_16', 4, 20),
        'ASIG_004_C2S1_E' : ('PROFESOR_16', 1, 20),
        'ASIG_038_C2S2_C' : ('PROFESOR_16', 4, 20),
        'ASIG_038_C2S2_D' : ('PROFESOR_16', 4, 20),
        'ASIG_038_C2S2_H' : ('PROFESOR_16', 4, 20),

        'ASIG_035_C1S1_G' : ('PROFESOR_40', 4, 20),
        'ASIG_035_C1S1_H' : ('PROFESOR_40', 4, 20),
        'ASIG_035_C2S1_D' : ('PROFESOR_40', 4, 20),
        'ASIG_035_C2S1_F' : ('PROFESOR_40', 4, 20),
        'ASIG_035_C2S1_H' : ('PROFESOR_40', 4, 20),

        'ASIG_079_C4S1_C' : ('PROFESOR_44', 4, 25),
        'ASIG_034_C4S1_C' : ('PROFESOR_44', 4, 30),
        'ASIG_035_C1S1_D' : ('PROFESOR_44', 4, 20),
        'ASIG_035_C1S1_F' : ('PROFESOR_44', 4, 20),
        'ASIG_080_C4S2_B' : ('PROFESOR_44', 4, 30),

        'ASIG_051_C2S2_DEF' : ('PROFESOR_57', 1, 25),
        'ASIG_038_C2S2_E' : ('PROFESOR_57', 4, 20),
        'ASIG_038_C2S2_F' : ('PROFESOR_57', 4, 20),
        'ASIG_089_C2S2_C' : ('PROFESOR_57', 9, 20),

        'ASIG_058_C1S1_D' : ('PROFESOR_62', 1, 20),
        'ASIG_044_C1S2_BCD' : ('PROFESOR_62', 1, 20),
        'ASIG_035_C1S2_D' : ('PROFESOR_62', 4, 20),
        'ASIG_035_C1S2_E' : ('PROFESOR_62', 4, 20),
        'ASIG_035_C1S2_F' : ('PROFESOR_62', 4, 20),
        'ASIG_035_C2S1_AB' : ('PROFESOR_62', 4, 25),

        'ASIG_035_C1S1_E' : ('PROFESOR_63', 4, 20),
        'ASIG_004_C1S1_E' : ('PROFESOR_63', 1, 20),
        'ASIG_035_C1S2_A' : ('PROFESOR_63', 4, 20),
        'ASIG_035_C1S2_B' : ('PROFESOR_63', 4, 25),
        'ASIG_095_C2S2_AB' : ('PROFESOR_63', 4, 25),

        'ASIG_006_C2S1_C' : ('PROFESOR_73', 1, 20),
        'ASIG_038_C2S2_G' : ('PROFESOR_73', 4, 20),

        'ASIG_105_C4S1_B' : ('PROFESOR_86', 4, 25),
        'ASIG_035_C1S1_A' : ('PROFESOR_86', 4, 20),
        'ASIG_106_C4S2_A' : ('PROFESOR_86', 4, 25),
        'ASIG_080_C4S2_C' : ('PROFESOR_86', 4, 30),

        'ASIG_105_C4S1_A' : ('PROFESOR_87', 4, 25),
        'ASIG_035_C1S1_B' : ('PROFESOR_87', 4, 20),
        'ASIG_035_C1S1_C' : ('PROFESOR_87', 4, 20),
        'ASIG_035_C2S1_C' : ('PROFESOR_87', 4, 20),
        'ASIG_035_C2S1_G' : ('PROFESOR_87', 4, 20),

        'ASIG_001_C1S1_A' : ('PROFESOR_01', 3, 20),
        'ASIG_002_C4S2_A' : ('PROFESOR_01', 3, 25),
        'ASIG_002_C4S2_B' : ('PROFESOR_01', 3, 25),
        'ASIG_002_C4S2_C' : ('PROFESOR_01', 3, 25),
}

# Diccionario para mapear las capacidades de las aulas
room_capacities = {
    0: 25,
    1: 25,
    2: 30,
    3: 20,
    4: 20,
    5: 20
}

Los siguientes datos servirán como base para el algoritmo genético que buscará optimizar la asignación de clases a aulas, profesores y estudiantes. En primer lugar, se definen parámetros generales como el número de aulas disponibles (`rooms`), el número de intervalos de tiempo (`slots`) en los que se pueden programar clases durante un día, y el número de días (`days`) a considerar para la programación. Además, se especifican parámetros "soft" (`LC_plus` y `SC_plus`), que determinan el número máximo de intervalos de tiempo en los que un profesor o un grupo de estudiantes puede estar en el centro educativo sin incurrir en un coste adicional. Finalmente, se crean dos diccionarios (`teacher_to_pref` y `student_to_pref`) que asignan a cada profesor y grupo de estudiantes un intervalo de tiempo preferido para tener clases.

In [ ]:
# Parámetros para la creación del horario (aulas, slots de tiempo, días a programar)
rooms = len(room_capacities)    # 5 en este caso
slots = 7
days = 5

# Parámetros "soft" para él cálculo de las restricciones suaves
LC_plus = 4     # número máximo de intervalos de tiempo que un profesor puede estar en el centro sin incurrir en un costo adicional
SC_plus = 2     # número máximo de intervalos de tiempo que un grupo de estudiantes puede estar en el centro en un día sin incurrir en un costo adicional

teacher_to_pref = {         # cada profesor tiene asignado un valor que representa su intervalo de tiempo preferido
    'PROFESOR_14': 2,       # Estos números representan índices de los intervalos de tiempo (slots) en los que prefieren dar clase
    'PROFESOR_16': 3,
    'PROFESOR_40': 1,
    'PROFESOR_44': 4,
    'PROFESOR_57': 0,
    'PROFESOR_62': 2,       # Ejemplo: al 'PROFESOR_62' le gustaría tener clases en el intervalo de tiempo 2 o antes
    'PROFESOR_63': 3,
    'PROFESOR_73': 1,
    'PROFESOR_86': 6,
    'PROFESOR_87': 0,
    'PROFESOR_01': 6,
}

student_to_pref = {         # Diccionario que mapea cada grupo de estudiantes a su intervalo de tiempo preferido
        'ASIG_034_C4S1_C' : 4,
        'ASIG_024_C3S1_B' : 4,
        'ASIG_035_C1S2_G' : 2,
        'ASIG_036_C1S2_H' : 6,
        'ASIG_035_C1S2_H' : 2,
        'ASIG_035_C2S1_E' : 0,
        'ASIG_004_C2S1_E' : 6,
        'ASIG_038_C2S2_C' : 1,
        'ASIG_038_C2S2_D' : 5,
        'ASIG_038_C2S2_H' : 0,
        'ASIG_035_C1S1_G' : 0,
        'ASIG_035_C1S1_H' : 1,
        'ASIG_035_C2S1_D' : 1,
        'ASIG_035_C2S1_F' : 4,
        'ASIG_035_C2S1_H' : 4,
        'ASIG_079_C4S1_C' : 0,
        'ASIG_034_C4S1_D' : 4,
        'ASIG_035_C1S1_D' : 0,
        'ASIG_035_C1S1_F' : 2,
        'ASIG_080_C4S2_B' : 4,
        'ASIG_051_C2S2_DEF' : 2,
        'ASIG_038_C2S2_E' : 0,
        'ASIG_038_C2S2_F' : 2,
        'ASIG_089_C2S2_C' : 6,
        'ASIG_058_C1S1_D' : 6,
        'ASIG_044_C1S2_BCD' : 6,
        'ASIG_035_C1S2_D' : 3,
        'ASIG_035_C1S2_E' : 3,
        'ASIG_035_C1S2_F' : 2,
        'ASIG_035_C2S1_AB' : 2,
        'ASIG_035_C1S1_E' : 2,
        'ASIG_004_C1S1_E' : 5,
        'ASIG_035_C1S2_A' : 0,
        'ASIG_035_C1S2_B' : 2,
        'ASIG_095_C2S2_AB' : 6,
        'ASIG_006_C2S1_C' : 4,
        'ASIG_038_C2S2_G' : 6,
        'ASIG_105_C4S1_B' : 0,
        'ASIG_035_C1S1_A' : 2,
        'ASIG_106_C4S2_A' : 0,
        'ASIG_080_C4S2_C' : 0,
        'ASIG_105_C4S1_A' : 0,
        'ASIG_035_C1S1_B' : 4,
        'ASIG_035_C1S1_C' : 4,
        'ASIG_035_C2S1_C' : 3, 
        'ASIG_035_C2S1_G' : 4,
        'ASIG_001_C1S1_A' : 6,
        'ASIG_002_C4S2_A' : 6,
        'ASIG_002_C4S2_B' : 6,
        'ASIG_002_C4S2_C' : 6,
}

## DEFINICIÓN DE CLASES, FUNCIONES Y LISTAS AUXILARES

El siguiente bloque de código prepara la estructura de datos que se utilizará para codificar y decodificar soluciones en el algoritmo genético de planificación de horarios. Primero, se crea una lista vacía llamada `lista_clases` que posteriormente se llena con tuplas que contienen información sobre las asignaturas: el profesor que la imparte, el nombre de la asignatura y la capacidad de la aula necesaria. Cada asignatura se repite un número de veces igual a las horas que necesita ser impartida, según lo especificado en el diccionario `asignaturas`. Seguido a esto, se genera una lista llamada `profesor_clase_a_entero`, la cual se utiliza para la codificación biunívoca en enteros de las clases en `lista_clases`. Finalmente, se crea un diccionario `id_tupla_AsigProf` que mapea cada índice entero a su correspondiente tupla de `lista_clases`, lo cual facilita la identificación y manipulación de las asignaturas durante el algoritmo de optimización.

In [ ]:
# Crear lista de clases a repartir en el horario a partir de 'asignaturas'
lista_clases = []
for subject, (professor, hours, capacity) in asignaturas.items():
    lista_clases.extend([(professor, subject, capacity)] * hours)

# Crear listas para la codificación biunívoca (en enteros)
profesor_clase_a_entero = list(range(len(lista_clases)))

# Crear diccionario para codificar cada asignatura según 'profesor_clase_a_entero'
id_tupla_AsigProf = {i: valor for i, valor in enumerate(lista_clases)}

Se definen varias funciones auxiliares diseñadas para simplificar la tarea de decodificar y manipular la información de las asignaturas y las aulas. Las primeras cuatro funciones (`profesor`, `asignatura`, `capacidad` y `tupla_profesor_asignatura`) toman un número entero como argumento y devuelven distintas partes de la tupla asociada a ese entero en el diccionario `id_tupla_AsigProf`. Estas funciones son útiles para obtener rápidamente los detalles relevantes de cada asignatura, como el profesor que la imparte o la capacidad del aula requerida. La última función, `aulas_adecuadas`, se utilizará más adelante para inicializar la población, y toma la capacidad necesaria para una asignatura como argumento y devuelve una lista de aulas que pueden acomodar dicha capacidad. Esta lista está ordenada según la proximidad entre la capacidad de la aula y la capacidad necesaria, lo cual facilita la selección de una aula óptima.

In [ ]:
# Para descodificar el entero y devolver los valores solo hay que usar estas funciones
def profesor(entero, id_tupla_AsigProf=id_tupla_AsigProf):
    return id_tupla_AsigProf[entero][0]

def asignatura(entero, id_tupla_AsigProf=id_tupla_AsigProf):
    return id_tupla_AsigProf[entero][1]

def capacidad(entero, id_tupla_AsigProf=id_tupla_AsigProf):
    return id_tupla_AsigProf[entero][2]

def tupla_profesor_asignatura(entero, id_tupla_AsigProf=id_tupla_AsigProf):
    return id_tupla_AsigProf[entero]

# Función para obtener las aulas ordenadas según su capacidad
def aulas_adecuadas(capacidad_necesaria, room_capacities=room_capacities):
    # Primero, filtramos las aulas que no pueden acomodar la capacidad necesaria
    aulas_posibles = {k: v for k, v in room_capacities.items() if v >= capacidad_necesaria}
    
    # Luego, ordenamos las aulas restantes según la diferencia entre la capacidad del aula y la capacidad necesaria
    aulas_ordenadas = sorted(aulas_posibles.keys(), key=lambda k: abs(aulas_posibles[k] - capacidad_necesaria))
    
    return aulas_ordenadas

Se van a definir unas funciones, que forman un conjunto de herramientas que permiten una gestión más eficiente y clara de la población dentro del algoritmo genético. Estas funciones facilitan la manipulación y transformación de matrices que representan poblaciones de horarios universitarios en un algoritmo genético.

La primera función, `flatten_population`, convierte una lista de matrices tridimensionales (cada una representando un individuo con un horario potencial) en una matriz bidimensional. Esta matriz 2D facilita el manejo de la población en cálculos subsecuentes, donde cada fila representa un individuo y cada columna un espacio único en el horario. 

La función `reconstruct_population` realiza la operación inversa: toma una matriz bidimensional y la convierte de nuevo en una lista de matrices 3D, cada una representando un horario único. Esto es útil para reconstruir la población después de operaciones como la selección, el cruce y la mutación.

La función `deflatten_individual` toma un vector plano que representa un único horario y lo convierte de nuevo en su formato original de matriz 3D. Esta función se podría considerar como un caso especial de `reconstruct_population` pero para un único individuo.

Finalmente, la función `reassign_population` toma una matriz de horarios "aplanada" y reemplaza cada ID único por una tupla que contiene el profesor y la asignatura correspondiente. Esta conversión facilita la interpretación y el análisis del horario, ya que proporciona un contexto más claro para cada entrada.

In [ ]:
##### Función que convierte una lista de matrices 3D en una matriz 2D, donde cada fila representa #####
##### un individuo y cada columna un espacio único en el horario, identificado con un ID único #####
def flatten_population(population, rooms=rooms, slots=slots, days=days):
    n_individuals = len(population)
    n_cells_per_individual = rooms * slots * days
    population_matrix = np.empty((n_individuals, n_cells_per_individual), dtype=int)
    population_matrix.fill(-1)

    # Llenar la matriz de población
    for i, individual in enumerate(population):
        flat_vector = []
        for room in individual:
            for slot in room:
                for day in slot:
                    flat_vector.append(day if day != -1 else -1)
        population_matrix[i, :] = flat_vector
    return population_matrix


##### Función que reconstruye la matriz población a partir de una matriz aplanada #####
def reconstruct_population(population_matrix, rooms=rooms, slots=slots, days=days):
    # Extraer las dimensiones de la matriz población
    n_individuals, n_cells_per_individual = population_matrix.shape
    # Crear una lista vacía para almacenar las poblaciones reconstruidas
    reconstructed_population = []

    # Iterar sobre cada individuo en la población
    for i in range(n_individuals):
        # Inicializar una matriz 3D vacía para un individuo
        individual_3D = np.empty((rooms, slots, days), dtype=object)
        individual_3D.fill(None)        # Llenamos inicialmente con None
        
        for room in range(rooms):       # Iterar sobre cada aula
            for slot in range(slots):   # Iterar sobre cada intervalo de tiempo
                for day in range(days): # Iterar sobre cada día
                    
                    # Calcular el índice correspondiente en la matriz aplanada
                    idx = room * (slots * days) + slot * days + day
                    # Asignar el valor del individuo aplanado a la matriz 3D
                    individual_3D[room, slot, day] = population_matrix[i, idx]
        
        # Agregar el individuo 3D reconstruido a la población reconstruida
        reconstructed_population.append(individual_3D)
    
    # Devolver la población reconstruida
    return reconstructed_population


##### Función que convierte una matriz 2D en una  3D, a partir del horario plano. Ejemplo: deflatten_population_matrix(population_matrix[0,:]) #####
def deflatten_individual(individual_flat, rooms=rooms, slots=slots, days=days):
    return individual_flat.reshape((rooms, slots, days))


##### Toma una matriz de horarios 3D y la modifica para que cada celda contenga una tupla (profesor, asignatura) en lugar de un ID único #####
def reassign_population(deflattened_population, lista_clases=lista_clases):    
    # Convertimos la matriz de numeros a profesores (o profesor asignatura si cambiamos la función por id_to_tuple)
    matriz_profesores = np.array([[lista_clases[x][:2] if x != -1 else None for x in fila] for fila in deflattened_population], dtype=object)
    return matriz_profesores

La siguiente función `guarda_resultados`, toma una matriz tridimensional que representa diferentes horarios de aulas y la convierte en un DataFrame de Pandas para luego guardarla en un archivo de Excel. La matriz se acomoda y se transforma en una serie de DataFrames, cada uno correspondiente a una "aula", y estos se concatenan en un DataFrame final. Este DataFrame final contiene columnas para 'Aula', 'Slot' (intervalo de tiempo), 'Día', y 'Población' (el índice del horario dentro de la matriz original). Finalmente, la función verifica si un archivo de Excel con el nombre "horario_aulas.xlsx" ya existe: si existe, añade una nueva hoja con los datos; si no, crea un nuevo archivo y guarda los datos en él.

La función `guardar_resultados_pymoo_en_excel` toma un objeto de resultados `res` generado por un algoritmo de optimización de la biblioteca pymoo y lo guarda en un archivo Excel. La función crea tres DataFrames: `df_X` para las variables de decisión, `df_F` para los valores objetivos y `df_G` para las restricciones, si existen. Cada DataFrame se guarda en una hoja diferente del archivo Excel, y el nombre de cada hoja incluye una marca de tiempo para facilitar la identificación. El archivo Excel se guarda con un nombre especificado por el argumento `nombre_archivo`, y si el archivo ya existe, la función añade nuevas hojas en lugar de sobrescribirlo.

In [ ]:
#################### FUNCIÓN PARA ARREGLAR EL DATAFRAME POPULATION Y GUARDAR LA POBLACIÓN EN EXCEL ####################

def guarda_resultados(matriz_final, hoja="horario_aulas_"):        # Acepta una matriz 3D para guaardar los resultados

    # Lista para almacenar los DataFrames de cada aula
    aula_dfs = []

    # Inicializa las listas para almacenar los valores de 'Slot' y 'Aula' de la primera población
    slots_por_dia_primera_poblacion = None
    aulas_primera_poblacion = None

    # Recorre las aulas (rooms)
    for room_index, room_schedule in enumerate(matriz_final):
        # Aplana el horario de esta aula en un DataFrame
        room_schedule_flat = room_schedule.reshape(-1, room_schedule.shape[2])
        
        # Crea un DataFrame a partir de la matriz plana
        df = pd.DataFrame(room_schedule_flat, columns=[f'Día {i + 1}' for i in range(room_schedule.shape[2])])
        
        # Si es la primera población, genera las columnas 'Slot' y 'Aula'
        if room_index == 0:
            num_filas = room_schedule_flat.shape[0]
            slots_por_dia_primera_poblacion = [(i % slots) + 1 for i in range(num_filas)]
            
            aula_actual = 0
            aulas_primera_poblacion = []
            for slot in slots_por_dia_primera_poblacion:
                if slot == 1:
                    aula_actual = (aula_actual % rooms) + 1
                aulas_primera_poblacion.append(f'Aula {aula_actual}')
        
        # Inserta las columnas en la posición deseada
        df.insert(0, 'Aula', aulas_primera_poblacion)
        df.insert(1, 'Slot', slots_por_dia_primera_poblacion)
            
        # Agrega una columna 'Población' con el número de la población
        df['Población'] = f'Población {room_index + 1}'
        
        # Agrega el DataFrame a la lista de DataFrames de aulas
        aula_dfs.append(df)

    # Concatena todos los DataFrames de aulas en uno solo
    final_df = pd.concat(aula_dfs, ignore_index=True)

    # Ajusta el orden de las columnas para que 'Población' aparezca al final
    column_order = [col for col in final_df.columns if col != 'Población'] + ['Población']
    final_df = final_df[column_order]

    # Luego, guarda este DataFrame en un archivo Excel
    nombre_archivo = "horario_aulas.xlsx"
    nombre_hoja = hoja + datetime.now().strftime("%Y%m%d%H%M%S")

    # Verifica si el archivo ya existe
    if os.path.exists(nombre_archivo):
        # Si existe, agrega una nueva hoja
        with pd.ExcelWriter(nombre_archivo, engine='openpyxl', mode='a') as writer:
            final_df.to_excel(writer, sheet_name=nombre_hoja, index=False)
    else:
        # Si no existe, crea un nuevo archivo y añade la hoja
        final_df.to_excel(nombre_archivo, sheet_name=nombre_hoja, index=False)


#################### FUNCIÓN PARA GUARDAR EN EXCEL LOS RESULTADOS DEL OBJETO RES DE PYMOO ####################

def guardar_resultados_pymoo_en_excel(res, nombre_archivo="resultados.xlsx"):
    """
    Guarda los resultados del algoritmo de pymoo en un archivo Excel.
    
    Parámetros:
        res: Objeto de resultado de pymoo.
        nombre_archivo (str, opcional): Nombre del archivo Excel donde se guardarán los resultados. Por defecto es "resultados.xlsx".
    """
    
    # Crear una marca de tiempo para el nombre de la hoja
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    
    # Crear un DataFrame para las soluciones
    if len(res.X.shape) == 1:  # Si res.X es un vector
        df_X = pd.DataFrame(res.X, columns=["Variable_1"])
    else:  # Si res.X es una matriz
        df_X = pd.DataFrame(res.X, columns=[f"Variable_{i+1}" for i in range(res.X.shape[1])])

    # Crear un DataFrame para los objetivos
    if len(res.F.shape) == 1:  # Si res.F es un vector
        df_F = pd.DataFrame(res.F, columns=["Objetivo_1"])
    else:  # Si res.F es una matriz
        df_F = pd.DataFrame(res.F, columns=[f"Objetivo_{i+1}" for i in range(res.F.shape[1])])

    # Guardar los DataFrames en un archivo Excel con la marca de tiempo
    with pd.ExcelWriter(nombre_archivo, mode='a') as writer:  # Modo 'a' para agregar al archivo si ya existe
        df_X.to_excel(writer, sheet_name=f"Soluciones_{timestamp}", index=False)
        df_F.to_excel(writer, sheet_name=f"Objetivos_{timestamp}", index=False)

        # Si hay restricciones, guárdalas también
        if hasattr(res, 'G') and res.G is not None:
            if len(res.G.shape) == 1:  # Si res.G es un vector
                df_G = pd.DataFrame(res.G, columns=["Restricción_1"])
            else:  # Si res.G es una matriz
                df_G = pd.DataFrame(res.G, columns=[f"Restricción_{i+1}" for i in range(res.G.shape[1])])
            df_G.to_excel(writer, sheet_name=f"Restricciones_{timestamp}", index=False)

Las funciones `initialize_population`, `initialize_random_population`, `initialize_mixed_population` y `initialize_special_population` son responsables de generar las poblaciones iniciales para el algoritmo genético que aborda el problema de planificación de horarios universitarios.

- **Función `initialize_population`**: esta función genera una población de horarios donde cada horario es una matriz 3D (aulas, franjas horarias, días). La función inicializa estos horarios de manera que cumplan con ciertas "restricciones fuertes" o "hard constraints". Durante la inicialización, se intenta asignar aulas de acuerdo con la capacidad necesaria para cada clase. A medida que se asignan aulas y tiempos, se invoca a la función `check_hard_constraints` para asegurar que la asignación cumpla con las restricciones fuertes. Si una asignación no las cumple, se revierte y se intenta de nuevo.

- **Función `initialize_random_population`**: a diferencia de `initialize_population`, esta función genera una población de horarios de forma completamente aleatoria. Aquí no se tienen en cuenta las restricciones, ni se busca asignar aulas más adecuadas en términos de capacidad. Simplemente se distribuyen las clases en los horarios de forma aleatoria.

- **Función `initialize_mixed_population`**: esta función combina los enfoques de las dos funciones anteriores para crear una población mixta. Se especifica un ratio (`hard_ratio`) que determina la proporción de la población inicial que se generará utilizando `initialize_population`. El resto se generará con `initialize_random_population`. La idea es beneficiarse tanto de horarios que ya cumplen con algunas restricciones como de la diversidad que ofrece una inicialización aleatoria.

- **Función `evaluate_population`**: está diseñada para calcular y devolver un conjunto de valores de "fitness" de cada individuo dentro de una población. Esta evaluación se realiza mediante la invocación de la función `check_soft_constraints`.

- **Función `initialize_special_population`**: se encarga de inicializar una población, pero con un enfoque particular: empieza creando y evaluando una población inicial (`initial_population_size`). Luego, ordena esta población inicial por su valor de aptitud y selecciona los `top_n` mejores individuos. El resto de la población hasta alcanzar el tamaño deseado (`population_size`) se compone de individuos generados de manera aleatoria. Esto significa que la población final contendrá una mezcla de los mejores individuos encontrados hasta el momento y nuevos individuos aleatorios, permitiendo tanto la explotación de buenas soluciones conocidas como la exploración de nuevas áreas del espacio de búsqueda.

Estas funciones proporcionan una base sólida para comenzar la optimización mediante algoritmos genéticos, permitiendo tanto cumplimiento de restricciones como diversidad en la población inicial.

In [ ]:
# Inicialización de la población cumpliendo restricciones (se tienen en cuenta las aulas más adecuadas)
def initialize_population(population_size, rooms=rooms, slots=slots, days=days, lista_clases=lista_clases, seed=None):
    if seed is not None:
        np.random.seed(seed)
        random.seed(seed)
        
    population = []
    
    for _ in range(population_size):
        horario = np.empty((rooms, slots, days), dtype=int)
        horario.fill(-1)
        profesor_clase_a_entero = list(range(len(lista_clases)))
        
        while profesor_clase_a_entero:
            candidate = random.choice(profesor_clase_a_entero)
            capacidad_necesaria = capacidad(candidate)  # Obtener la capacidad necesaria del candidato
            
            # Este enfoque permite rellenar las aulas más adecuadas a la capacidad del evento en cuestión
            aulas_orden = aulas_adecuadas(capacidad_necesaria, room_capacities)

            for aula in aulas_orden:
                lugares_vacios = [(s, d) for s in range(slots) for d in range(days) if horario[aula, s, d] == -1]
                if lugares_vacios:
                    s, d = random.choice(lugares_vacios)
                    horario[aula, s, d] = candidate
                    if check_hard_constraints(horario):
                        profesor_clase_a_entero.remove(candidate)
                        break
                    else:
                        horario[aula, s, d] = -1  # Si no cumple con las restricciones, vuelve a ponerlo como vacío

        population.append(horario)

    return population


# Inicialización de la población de forma completamente aleatoria
def initialize_random_population(random_population_size, rooms=rooms, slots=slots, days=days, lista_clases=lista_clases):
    random_population = []
    
    for _ in range(random_population_size):
        # Crear el array inicial
        # Primero, llenamos con números entre 0 y len(clases)-1
        initial_elements = np.arange(len(lista_clases))

        # Luego, llenamos el resto con -1
        remaining_elements = rooms*slots*days - len(lista_clases)
        remaining_array = np.full(remaining_elements, -1)

        # Unir los dos arrays para formar el array completo
        full_array = np.concatenate((initial_elements, remaining_array))

        # Mezclar aleatoriamente los elementos
        np.random.shuffle(full_array)

        # Si necesitas, puedes darle forma de (rooms, slots, days)
        horario = full_array.reshape((rooms, slots, days))

        random_population.append(horario)
    
    return random_population

# Inicialización mixta de la población
def initialize_mixed_population(population_size, hard_ratio=0.7, seed=None):
    hard_population_size = int(population_size * hard_ratio)
    random_population_size = population_size - hard_population_size
    
    hard_population = initialize_population(hard_population_size, seed=seed)
    random_population = initialize_random_population(random_population_size)
    
    return hard_population + random_population

# Dada una población, esta función evalúa la población basada en 'check_soft_constraints'
def evaluate_population(population):
    return [check_soft_constraints(individuo, cond=True) for individuo in population]

# Incializa la población con los 'top_n' mejores individuos de 'initial_population_size', y el resto aleatorios hasta completar 'population_size'
def initialize_special_population(population_size=200, top_n=1, initial_population_size=300, seed=None):
    # Paso 1: Crear y evaluar una población inicial mayor
    initial_population = initialize_population(initial_population_size, seed=seed)
    fitness_values = evaluate_population(initial_population)

    # Paso 2: Ordenar la población inicial por fitness (menor es mejor)
    sorted_indices = np.argsort(fitness_values)

    # Paso 3: Seleccionar los N individuos con mejor fitness
    best_individuals = [initial_population[i] for i in sorted_indices[:top_n]]

    # Paso 4: Crear el resto de individuos aleatoriamente
    random_population = initialize_random_population(population_size - top_n)

    # Paso 5: Agregar ambos grupos de individuos a la población final
    final_population = best_individuals + random_population

    return final_population

### Tasa de Mutación Adaptativa

El siguiente código define una estrategia de adaptación dinámica para la tasa de mutación en un algoritmo genético. Utiliza variables globales `prev_avg_cv` para almacenar la violación de restricciones previa promedio y `stagnation_counter` para contar cuántas generaciones han pasado sin mejoras significativas en el rendimiento.

La función `adaptive_mutation` ajusta dinámicamente la tasa de mutación basada en el rendimiento del algoritmo. Si el algoritmo se estanca, es decir, no mejora durante un cierto número de generaciones (`STAGNATION_GEN_COUNT`), la tasa de mutación se duplica. Esto permite una mayor exploración del espacio de soluciones para superar el estancamiento. Una vez que el rendimiento mejora, la tasa de mutación se restablece a su valor original. El umbral de detección de estancamiento es controlado por `STAGNATION_THRESHOLD`.

La clase `AdaptiveMutationCallback` se implementa como un Callback en el ciclo de vida del algoritmo genético. Este Callback se invoca en cada generación del algoritmo, donde ajusta la tasa de mutación de acuerdo con el desempeño del algoritmo actual. Utiliza la función `adaptive_mutation` para realizar este ajuste y luego aplica la nueva tasa de mutación al algoritmo.

Esta estrategia es especialmente útil en problemas complejos donde el rendimiento del algoritmo puede estancarse en mínimos locales. El ajuste dinámico de la tasa de mutación permite una mejor exploración y explotación del espacio de búsqueda.

In [ ]:
prev_avg_cv = None
stagnation_counter = 0
original_mutation_rate = 0.005  # Asumimos este % como tasa de mutación inicial para AG y NSGA2

def adaptive_mutation(current_avg_cv, mutation_rate):
    global prev_avg_cv, stagnation_counter, original_mutation_rate
    
    # Umbral de cambio mínimo para detectar estancamiento
    STAGNATION_THRESHOLD = 0.01
    # Número de generaciones sin cambios significativos para considerar estancamiento
    STAGNATION_GEN_COUNT = 5
    
    # Si es la primera vez, establece prev_avg_cv
    if prev_avg_cv is None:
        prev_avg_cv = current_avg_cv
        return mutation_rate
    
    # Comprobar si el algoritmo se ha estancado
    if abs(current_avg_cv - prev_avg_cv) < STAGNATION_THRESHOLD:
        stagnation_counter += 1
    else:
        stagnation_counter = 0  # Resetear contador si hay mejora
    
    # Si hay estancamiento durante STAGNATION_GEN_COUNT generaciones
    if stagnation_counter >= STAGNATION_GEN_COUNT:
        # Duplicar la tasa de mutación
        mutation_rate *= 2
        # Limitar la tasa de mutación a un máximo (50% en este caso)
        mutation_rate = min(mutation_rate, 0.5)
    
    # Si el algoritmo mejora después de aumentar la tasa de mutación
    if mutation_rate > original_mutation_rate and abs(current_avg_cv - prev_avg_cv) >= STAGNATION_THRESHOLD:
        mutation_rate = original_mutation_rate  # Restablecer tasa de mutación
    
    prev_avg_cv = current_avg_cv
    return mutation_rate

class AdaptiveMutationCallback(Callback):

    def __init__(self):
        super().__init__()
        self.prev_avg_cv = float("inf")
        self.mutation_rate = original_mutation_rate

    def __call__(self, algorithm):
        # Extraer la métrica "cv (avg)" de la población actual
        current_avg_cv = algorithm.pop.get("CV").mean()

        # Actualizar la tasa de mutación usando adaptive_mutation
        self.mutation_rate = adaptive_mutation(current_avg_cv, self.mutation_rate)

        # Asumiendo que la mutación tiene un atributo 'prob' para la tasa de mutación, actualizamos este valor aquí:
        algorithm.mutation.prob = self.mutation_rate

        # Actualizamos prev_avg_cv para la próxima generación
        self.prev_avg_cv = current_avg_cv

# RESTRICCIONES DURAS

La planificación de horarios universitarios es una tarea compleja que implica asignar recursos limitados (como profesores, salas y equipos) a una serie de eventos (como clases, seminarios y exámenes) de manera que se cumplan un conjunto de restricciones. En este notebook, se presentan funciones para verificar cuatro restricciones "hard", es decir, condiciones que deben cumplirse sin excepción para que una solución sea considerada factible.

### Restricción dura 01: `check_professor_multiple_rooms` "Un profesor no puede estar en dos aulas a la vez"
Esta función toma como entrada una matriz tridimensional que representa la asignación de eventos a salas en diferentes slots de tiempo y días de la semana. Devuelve el número de veces que un profesor ha sido asignado a más de una sala en el mismo slot de tiempo y el mismo día, lo cual sería una violación de la restricción.

### Restricción dura 02: `check_package_multiple_events` "Un paquete no puede tener múltiples eventos"
Esta función verifica si se han programado múltiples eventos del mismo tipo en un solo slot de tiempo y día, lo que resultaría en una violación de la restricción.

### Restricción dura 03: `check_room_capacity` "La sala debe tener la capacidad adecuada"
Esta función verifica si la capacidad de cada sala es suficiente para el evento programado. Si la capacidad del evento es mayor que la de la sala, se considera una violación de la restricción.

### Restricción dura 04: `verificar_cumplimiento` "Verificar que todo el horario está asignado"
Esta función verifica si el horario contiene el número correcto de asignaturas y si cada asignatura aparece solo una vez. De no ser así, devuelve la diferencia.

In [ ]:
#######################################################################################################
##### RESTRICCIÓN HARD 01 #################### "Un profesor no puede estar en dos aulas a la vez" #####
#######################################################################################################

def check_professor_multiple_rooms(individual_3D):
    violations = 0                              # Inicializar el conteo de violaciones
    rooms, slots, days = individual_3D.shape    # Obtener las dimensiones de la matriz 3D
    
    for slot in range(slots):                   # Iterar a través de los slots de tiempo
        for day in range(days):                 # Iterar a través de los días de la semana
            
            professors_in_slot_day = []         # Lista para almacenar los profesores que están dando clases en este slot y día
            
            for room in range(rooms):           # Iterar a través de las aulas
                entry = individual_3D[room, slot, day]      # Obtener la entrada asignada a esta combinación de slot, día y aula
                
                if entry >= 0:                                   # Si hay una entrada
                    professor = profesor(entry)             # Descomponer la entrada en profesor y asignatura
                    
                    if professor in professors_in_slot_day: # Si el profesor ya está en la lista, hay una violación
                        violations += 1                     # Incrementar el conteo de violaciones
                    else:
                        professors_in_slot_day.append(professor)    # Añadir el profesor a la lista de profesores para este slot y día

    return violations                           # Devolver el número total de violaciones de esta restricción

##################################################################################################
##### RESTRICCIÓN HARD 02 #################### "Un paquete no puede tener múltiples eventos" #####
##################################################################################################

def check_package_multiple_events(individual_3D):
    violations = 0
    rooms, slots, days = individual_3D.shape
    
    for slot in range(slots):                  
        for day in range(days):                 
            seen_entries = set()                # Conjunto para almacenar los valores de entry que hemos visto en este slot y día
            
            for room in range(rooms):
                entry = individual_3D[room, slot, day]
                
                if entry != -1:                 # Si hay un evento programado en este slot, día, y aula (se asume -1 como un vacío en el horario)
                    if entry in seen_entries:   # Si ya hemos visto este valor de entry antes
                        violations += 1         # Incrementar el contador de violaciones
                    else:
                        seen_entries.add(entry) # Agregar el valor de entry al conjunto de valores vistos
    
    return violations                           # Devolver el número total de violaciones de esta restricción

###############################################################################################
##### RESTRICCIÓN HARD 03 #################### "La sala debe tener la capacidad adecuada" #####
###############################################################################################

def check_room_capacity(individual_3D):
    violations = 0
    rooms, slots, days = individual_3D.shape
    
    for slot in range(slots):
        for day in range(days):
            for room in range(rooms):
                
                entry = individual_3D[room, slot, day]
                
                if entry >= 0:                                  # Si hay una entrada
                    event_capacity = capacidad(entry)           # Obtener la capacidad requerida para el evento
                    room_capacity = room_capacities[room]               # Obtener la capacidad de la sala
                    
                    if event_capacity > room_capacity:                 # Si la capacidad requerida es mayor que la de la sala
                        violations += 1                                # Incrementar el conteo de violaciones (mejora: PROBAR CON abs(event_capacity - room_capacity))
    
    return violations                           # Devolver el número total de violaciones de esta restricción

##################################################################################################
##### RESTRICCIÓN HARD 04 #################### "Verificar que todo el horario está asignado" #####
##################################################################################################

def verificar_cumplimiento(individual_2D, lista_clases=profesor_clase_a_entero):
    """
    Verifica si el horario contiene el número correcto de asignaturas y si cada asignatura aparece solo una vez.
    El horario debe ser una matriz 2D aplanada.
    Retorna 0 si la condición se cumple, caso contrario, devuelve la diferencia.
    """
    # Crea un diccionario con el número de veces que cada asignatura debe aparecer
    clases_unicas = set(lista_clases)
    dict_clases = {clase: 1 for clase in clases_unicas}  # Ajusta el '1' según la frecuencia de cada asignatura-grupo

    # Cuenta el número de veces que cada asignatura aparece en el horario
    conteo_clases = Counter(individual_2D[individual_2D != -1])

    # Calcula la diferencia entre la cantidad esperada y la cantidad real para cada asignatura
    diferencia = 0
    for clase, conteo_esperado in dict_clases.items():
        conteo_real = conteo_clases.get(clase, 0)
        diferencia += abs(conteo_esperado - conteo_real)

    return diferencia  # Debe ser 0 para que la solución sea factible

La siguiente función, `check_hard_constraints`, evalúa si un horario dado viola o no ciertas restricciones *duras* (*hard constraints*). La función suma el número total de violaciones para diferentes tipos de restricciones: asignación de múltiples aulas a un mismo profesor (`check_professor_multiple_rooms`), asignación de múltiples eventos a un mismo paquete de tiempo (`check_package_multiple_events`), verificación general del cumplimiento de las restricciones (`verificar_cumplimiento`), y comprobación de la capacidad del aula (`check_room_capacity`). La variable `comprueba_poblacion` permite activar o desactivar algunas de estas comprobaciones. Si el número total de violaciones es cero, la función devuelve `True`, indicando que el horario es válido; de lo contrario, devuelve `False`.

In [ ]:
def check_hard_constraints(horario, comprueba_poblacion=False):
    # Sumar el número total de violaciones de cada restricción.
    total_violations = 0
    total_violations += check_professor_multiple_rooms(horario)
    if comprueba_poblacion:     # El True en 'comprueba_poblacion' hace que esto se ejecute solamente para comprobación, no en la inicialización del horario
        total_violations += check_package_multiple_events(horario)
        total_violations += verificar_cumplimiento([flatten_population(horario)])
    total_violations += check_room_capacity(horario)

    # Si el número total de violaciones es cero, entonces el horario es válido (devuelve True).
    return total_violations == 0

# RESTRICCIONES SUAVES

La planificación de horarios en un entorno universitario no sólo implica cumplir con ciertas restricciones obligatorias ("*hard*"), sino que también busca optimizar ciertos aspectos cualitativos a través de restricciones "*suaves*" ("*soft*"). Estas restricciones suaves son flexibles y se pueden violar a cambio de cierto "*coste*". En este notebook se implementan funciones para calcular el coste asociado a cuatro restricciones suaves diferentes.

### Coeficientes de ponderación
Los coeficientes `alpha1, alpha2, alpha3, alpha4` son utilizados para dar peso a cada una de las restricciones suaves en la función objetivo del algoritmo genético.

### Restricción suave 01: `cost_teacher_load` "Minimizar el tiempo que los profesores pasan en el centro"
Esta función evalúa el tiempo que los profesores pasan en el centro universitario. Utiliza una variable `LC_plus` para definir un límite aceptable de tiempo. Si un profesor supera este límite, el coste aumenta.

### Restricción suave 02: `cost_teacher_time_preference` "Asignación de tiempo preferido para profesores"
Esta función evalúa si los profesores han sido asignados a sus slots de tiempo preferidos. El coste aumenta cada vez que un profesor es asignado fuera de su tiempo preferido.

### Restricción suave 03: `cost_student_load` "Distribución de carga para estudiantes"
Esta función evalúa la distribución de la carga académica para los grupos de estudiantes. Utiliza una variable `SC_plus` para definir un límite aceptable de carga. Si un grupo de estudiantes supera este límite, el coste aumenta.

### Restricción suave 04: `cost_student_time_preference` "Asignación de tiempo preferido para alumnos"
Esta función evalúa si los grupos de estudiantes han sido asignados a sus slots de tiempo preferidos. El coste aumenta cada vez que un grupo de estudiantes es asignado fuera de su tiempo preferido.

In [ ]:
# Coeficientes de ponderación para dar peso a las restricciones suaves (cada institución debe valorar los suyos)
alpha1, alpha2, alpha3, alpha4 = 1, 1, 1, 1

################################################################################################################
##### RESTRICCIÓN SOFT 01 #################### "Minimizar el tiempo que los profesores pasan en el centro" #####
################################################################################################################

def cost_teacher_load(individual_3D, LC_plus=LC_plus):
    cost = 0  # Inicializar el coste
    rooms, slots, days = individual_3D.shape  # Obtener las dimensiones
    
    for day in range(days):
        teacher_event_count = {}  # Contar eventos por profesor en este día
        
        for slot in range(slots):
            for room in range(rooms):
                entry = individual_3D[room, slot, day]
                if entry >= 0:
                    teacher = profesor(entry)
                    teacher_event_count[teacher] = teacher_event_count.get(teacher, 0) + 1
        
        for count in teacher_event_count.values():
            if count > LC_plus:
                cost += count - LC_plus
    
    scaled_cost = 10 * cost

    return scaled_cost

#####################################################################################################
##### RESTRICCIÓN SOFT 02 #################### "Asignación de tiempo preferido para profesores" #####
#####################################################################################################

def cost_teacher_time_preference(individual_3D, teacher_to_pref=teacher_to_pref):
    cost = 0  # Inicializar el coste
    rooms, slots, days = individual_3D.shape  # Obtener las dimensiones
    
    for day in range(days):
        for slot in range(slots):
            for room in range(rooms):
                entry = individual_3D[room, slot, day]
                if entry >= 0:
                    teacher = profesor(entry)
                    pref_slot = teacher_to_pref[teacher]
                    if slot >= pref_slot:
                        cost += 1
    
    return cost

#############################################################################################
##### RESTRICCIÓN SOFT 03 #################### "Distribución de carga para estudiantes" #####
#############################################################################################

def cost_student_load(individual_3D, SC_plus=SC_plus):
    cost = 0  # Inicializar el coste
    rooms, slots, days = individual_3D.shape  # Obtener las dimensiones
    
    for day in range(days):
        student_event_count = {}  # Contar eventos por grupo de estudiantes en este día
        
        for slot in range(slots):
            for room in range(rooms):
                entry = individual_3D[room, slot, day]
                if entry >= 0:
                    student_group = asignatura(entry)
                    student_event_count[student_group] = student_event_count.get(student_group, 0) + 1
        
        for count in student_event_count.values():
            if count > SC_plus:
                cost += count - SC_plus
    
    scaled_cost = 10 * cost

    return scaled_cost

##################################################################################################
##### RESTRICCIÓN SOFT 04 #################### "Asignación de tiempo preferido para alumnos" #####
##################################################################################################

def cost_student_time_preference(individual_3D, student_to_pref=student_to_pref):
    cost = 0  # Inicializar el coste
    rooms, slots, days = individual_3D.shape  # Obtener las dimensiones
    
    for day in range(days):
        for slot in range(slots):
            for room in range(rooms):
                entry = individual_3D[room, slot, day]
                if entry >= 0:
                    student_group = asignatura(entry)
                    pref_slot = student_to_pref[student_group]
                    if slot >= pref_slot:
                        cost += 1
    
    return cost 

La función `check_soft_constraints` evalúa el *coste* asociado a la violación de restricciones *suaves* (*soft constraints*) en un horario dado. Calcula cuatro diferentes tipos de costes: el coste asociado a la carga de trabajo de los profesores (`cost_teacher_load`), el coste relacionado con la preferencia de tiempo de los profesores (`cost_teacher_time_preference`), el coste debido a la distribución de carga para los estudiantes (`cost_student_load`), y el coste vinculado a la preferencia de tiempo para los estudiantes (`cost_student_time_preference`). Cada uno de estos costes se multiplica por su coeficiente de ponderación (`alpha1`, `alpha2`, `alpha3`, `alpha4`) para ajustar su importancia relativa. El coste total es la suma de estos cuatro costes ponderados.

La función tiene la opción de devolver el coste total (y los costes individuales si se 'descomentan') si se establece la variable `cond` como `True`. De lo contrario, devuelve una cadena de texto que describe el coste total y los costes individuales de cada tipo de restricción suave.

In [ ]:
def check_soft_constraints(horario, cond=False, alpha1=alpha1, alpha2=alpha2, alpha3=alpha3, alpha4=alpha4):
    # Sumar el número total de violaciones de cada restricción.
    cost, cost1, cost2, cost3, cost4 = 0, 0, 0, 0, 0
    cost1 = alpha1 * cost_teacher_load(horario)
    cost2 = alpha2 * cost_teacher_time_preference(horario)
    cost3 = alpha3 * cost_student_load(horario)
    cost4 = alpha4 * cost_student_time_preference(horario)
    cost = cost1 + cost2 + cost3 + cost4
    # Devuelve el coste 
    if cond:
        return cost #, cost1, cost2, cost3, cost4
    return "Coste total: {:.1f} - Coste SC1: {:.1f}. Coste SC2: {:.1f}. Coste SC3: {:.1f}. Coste SC4: {:.1f}.".format(cost, cost1, cost2, cost3, cost4)

# INICIALIZACIÓN DE LA POBLACIÓN

Se genera una población inicial mixta para el algoritmo genético. El tamaño de la población se define a través de la variable `population_size`. La variable `hard_ratio` se define como la inversa del tamaño de la población, lo cual significa que solo una de las soluciones generadas será mediante el método que cumple con las restricciones duras (`initialize_population`), mientras que el resto será generado de manera completamente aleatoria (`initialize_random_population`). Finalmente, la función `initialize_mixed_population` es invocada con estos parámetros para obtener la población inicial mixta, que se almacena en la variable `population`. Por otro lado, podemos generar la población inicial con `initialize_special_population`, que elige a los *n* mejores individuos de una población que cumple con las HC, y el resto generea individuos completamente aleatorios.

In [ ]:
# Generar la población mixta
semilla=1
population_size = 200
# hard_ratio = 1 / population_size
# population = initialize_mixed_population(population_size, hard_ratio, seed=semilla)

# Crear una población especial con el mejor fitess que cumple HC y el resto aleatorias para definir variedad
# Ejemplo de uso: top_n=2 mejores individuos de una población inicial de initial_population_size=300, y el resto aleatorios hasta llegar a 200
population = initialize_special_population(population_size=population_size, top_n=3, initial_population_size=300, seed=semilla)

## Arreglar y comprobar la matriz 'population'

Se crea una matriz *aplanada* a partir de la población inicial de horarios universitarios, utilizando la función `flatten_population`. El propósito de este aplanamiento es convertir la estructura de datos de la población, en una forma que sea compatible con la biblioteca Pymoo. Una vez aplanada, la población se almacena en la variable `population_matrix`, lista para ser utilizada en etapas posteriores del algoritmo.

In [ ]:
# Crear matriz aplanada de población para pymoo
population_matrix = flatten_population(population)

Ahora se realizan dos operaciones principales sobre la población de horarios. Primero, se utiliza la función `reassign_population` para convertir los enteros en la matriz aplanada `population_matrix` en sus correspondientes tuplas, que representan información más detallada sobre las asignaturas y profesores. Esta matriz transformada se almacena en `matriz_profesores`. Luego, la función `reconstruct_population` se emplea para volver a convertir esta matriz en una matriz 3D, recuperando así su estructura original de múltiples dimensiones. La matriz 3D resultante se guarda en la variable `matriz_final`, que estaría lista para análisis o visualización adicionales.

In [ ]:
matriz_profesores = reassign_population(population_matrix)      # Convertimos los enteros (en la matriz plana) en las tuplas correspondientes
matriz_final = reconstruct_population(matriz_profesores)        # Volvemos a convertir el horario en una matriz 3D

Este fragmento de código se utiliza para comprobación de la población inicial, e itera a través de cada elemento en la lista `population`. Utiliza la función `check_soft_constraints` para evaluar cada horario en términos de restricciones suaves y luego imprime el resultado de la evaluación.

In [ ]:
# Se comprueban las soft constraints para cada elemento de la población inicializada, 'population'
for i in range(len(population)):
    print(str(i)+": " + check_soft_constraints(population[i]))

## Guardar resultados en Excel

Se llama a la función `guarda_resultados` y le pasa como argumento `matriz_final`. La función guarda los resultados contenidos en `matriz_final` en un archivo Excel.

In [ ]:
# guarda_resultados(matriz_final)

# ALGORITMO GENÉTICO CON PYMOO 0.4.2

Se prepara la población inicial para ser utilizada en el framework de algoritmos genéticos Pymoo. Crea un nuevo objeto `Population` e inicializa su atributo *X* con `population_matrix`. Este atributo *X* usualmente almacena las soluciones candidatas del algoritmo genético. En este caso, `population_matrix` sería una representación matricial de todos los individuos de la población inicial. El código comentado inicializa la población utilizando la matriz relacionada con el 'fitness' de cada individuo.

In [ ]:
# Preparar la población inicial para introducirla en Pymoo
initial_population = Population.new("X", population_matrix)

# Variables globales o atributos de clase (para mutación adaptativa)
prev_avg_cv = None
stagnation_counter = 0
original_mutation_rate = 0.005  # Asumimos este % como tasa de mutación inicial para AG y NSGA2

# Crear una instancia del Callback
callback = AdaptiveMutationCallback()

# PARÁMETROS PARA LOS ALGORITMOS GENÉTICOS
pop_size = 100

# crossover = get_crossover("int_exp")
# crossover = get_crossover("int_one_point")
# crossover = get_crossover("int_two_point")
crossover = get_crossover("int_ux")
# crossover = get_crossover("int_hux")
# crossover = get_crossover("int_sbx", prob=0.35)
# crossover = get_crossover("int_k_point", n_points=20)

mutation = get_mutation("int_pm", prob=original_mutation_rate, eta=15)

# selection = get_selection("random")

# Número máximo de generaciones para detener el algoritmo
n_gen_max = 1000

# Semilla para reproducibilidad
seed = 19

# Criterios de detención del algoritmo
termination_nsga2 = ('n_gen', n_gen_max)
termination_ga = ('n_gen', n_gen_max)

# termination_nsga2 = MultiObjectiveDefaultTermination(
#     x_tol=1e-8,
#     cv_tol=1e-6,
#     f_tol=0.0025,
#     nth_gen=10,
#     n_last=200,
#     n_max_gen=1000,
#     n_max_evals=None
# )

# termination_ga = SingleObjectiveDefaultTermination(
#     x_tol=1e-8,
#     cv_tol=1e-6,
#     f_tol=1e-6,
#     nth_gen=10,
#     n_last=200,
#     n_max_gen=1000,
#     n_max_evals=None
# )

## Implementación del Algoritmo NSGA-II

El siguiente código implementa una versión del algoritmo NSGA-II (Non-dominated Sorting Genetic Algorithm II) específicamente adaptada para resolver el problema de planificación de horarios universitarios con dos objetivos. La clase `HorarioProblemNSGA2` hereda de la clase `Problem` de la biblioteca Pymoo y define el espacio de soluciones, las funciones objetivo y las restricciones del problema.

- **Espacio de Soluciones**: el número de variables de decisión corresponde al producto del número de aulas, franjas horarias y días. Los valores de estas variables son enteros que representan diferentes asignaturas y profesores. 

- **Funciones Objetivo**: se definen dos objetivos para minimizar: `f1` y `f2`. `f1` se enfoca en minimizar las penalizaciones relacionadas con las preferencias de los estudiantes, mientras que `f2` se centra en las preferencias de los profesores.

- **Restricciones**: hay cuatro restricciones duras que se evalúan para cada solución candidata. Estas restricciones incluyen que un profesor no pueda estar en múltiples aulas al mismo tiempo, que los paquetes de eventos se asignen correctamente, que se respeten las capacidades de las aulas y que se cumpla con otro conjunto de requisitos específicos.

Una vez definido el problema, se configura el algoritmo NSGA-II mediante la clase `NSGA2`. Se establece el tamaño de la población, el operador de cruce, el operador de mutación y otros parámetros relevantes.

Finalmente, se ejecutará el algoritmo mediante la función `minimize`, que lleva a cabo el proceso de optimización multiobjetivo. Se guarda el historial de la optimización y se devuelve la mejor solución encontrada (`mejor_solucion_nsga2`).

Este enfoque ofrece un marco robusto para la planificación de horarios, al considerar múltiples objetivos y restricciones, y adaptarse dinámicamente a lo largo del proceso de optimización.

In [ ]:
# DOBLE OBJETIVO: NSGA-II

# Clase para definir el problema
class HorarioProblemNSGA2(Problem):

    def __init__(self):
        super().__init__(n_var=rooms * slots * days,                                # Número de variables de decisión
                         n_obj=2,                                                   # Dos objetivos: minimizar penalizaciones
                         n_constr=4,                                                # Número de restricciones duras
                         xl=np.array([-1]*(rooms * slots * days)),                   # Valor mínimo de las variables de decisión
                         xu=np.array([len(lista_clases)-1]*(rooms * slots * days))  # Valor máximo de las variables de decisión                        
                        )

    def _evaluate(self, X, out, *args, **kwargs):

        # Inicializar las matrices que contendrán los valores de la función objetivo y restricciones
        f1 = np.zeros(X.shape[0])
        f2 = np.zeros(X.shape[0])
        g = np.zeros((X.shape[0], self.n_constr))

        # Bucle para evaluar cada individuo en la población
        for i in range(X.shape[0]):

            # Extraer el individuo actual y convertirlo a matriz 3D
            individual_3D = deflatten_individual(X[i])

            # Evaluar restricciones duras
            g[i, 0] = check_professor_multiple_rooms(individual_3D)
            g[i, 1] = check_package_multiple_events(individual_3D)
            g[i, 2] = check_room_capacity(individual_3D)
            g[i, 3] = verificar_cumplimiento(X[i])

            # Calculando f1: centrado en las preferencias de los estudiantes
            f1[i] = (alpha1 * cost_student_load(individual_3D, SC_plus) +
                    alpha2 * cost_student_time_preference(individual_3D, student_to_pref))

            # Calculando f2: suma ponderada de restricciones relacionadas con los profesores
            f2[i] = (alpha3 * cost_teacher_load(individual_3D, LC_plus) +
                    alpha4 * cost_teacher_time_preference(individual_3D, teacher_to_pref))

        # Asignar los resultados al objeto 'out'
        out["F"] = np.column_stack([f1, f2])
        out["G"] = g

# Instancia del problema
problem_nsga2 = HorarioProblemNSGA2()

# Configuración del algoritmo
algorithm_nsga2 = NSGA2(
    pop_size=pop_size,
    sampling=initial_population,
    crossover=crossover,
    mutation=mutation,
    # selection=selection,
    eliminate_duplicates=True
)

# Optimizar
res_nsga2 = minimize(problem_nsga2,
                    algorithm_nsga2,
                    termination=termination_nsga2,
                    save_history=True,
                    verbose=True,
                    seed=seed,
                    callback=callback)

# Obtener la mejor solución
mejor_solucion_nsga2 = res_nsga2.X

## Implementación del Algoritmo Genético (GA)

Este código representa una implementación de un algoritmo genético (GA, por sus siglas en inglés) para resolver un problema de planificación de horarios universitarios con un único objetivo de optimización. Aquí se detallan sus principales componentes:

#### Clase `HorarioProblemGA`

Esta clase deriva de la clase `Problem` del paquete Pymoo y se personaliza para adaptarse al problema específico de planificación de horarios:

- **Variables de decisión (`n_var`)**: representan las asignaciones entre profesores, asignaturas, salas y franjas horarias. Están determinadas por el producto de habitaciones (`rooms`), franjas horarias (`slots`) y días (`days`).
  
- **Objetivos (`n_obj`)**: se tiene un único objetivo, que es minimizar las penalizaciones asociadas a la planificación de horarios.

- **Restricciones (`n_constr`)**: el problema incluye 4 restricciones duras, como son la ocupación múltiple de aulas por un profesor, eventos múltiples para un mismo paquete de asignaturas, capacidad de las aulas y cumplimiento de ciertas condiciones no especificadas (`verificar_cumplimiento`).

- **Límites de las variables (`xl`, `xu`)**: los límites inferior y superior para cada variable de decisión se establecen en base a la lista de clases (`lista_clases`).

#### Método `_evaluate`

Esta función evalúa la aptitud de cada individuo en la población (`X`) y calcula las restricciones. El método hace lo siguiente:

1. **Inicialización**: se inicializan matrices para almacenar los valores de la función objetivo (`f1`) y las restricciones (`g`).

2. **Bucle sobre individuos**: para cada individuo en la población se realizan los siguientes pasos:
    - Se convierte el individuo a una matriz tridimensional (`individual_3D`).
    - Se evalúan las restricciones duras (`g`).
    - Se calcula el valor de la función objetivo (`f1`), que se basa en cuatro componentes diferentes: carga de estudiantes, preferencia temporal de los estudiantes, carga de los profesores y preferencia temporal de los profesores.

#### Instancia del problema y algoritmo

Se crea una instancia del problema (`problem_ga`) y se configura el algoritmo genético (`algorithm_ga`) con los parámetros deseados. Finalmente, se ejecuta el algoritmo de optimización, se guarda el historial y se obtiene la mejor solución (`mejor_solucion_ga`).

In [ ]:
# OBJETIVO SIMPLE: GA
# Clase para definir el problema
class HorarioProblemGA(Problem):

    def __init__(self):
        super().__init__(n_var=rooms * slots * days,                                # Número de variables de decisión (duplas asignatura profesor)
                         n_obj=1,                                                   # Un objetivo: minimizar penalizaciones
                         n_constr=4,                                                # Número de restricciones duras
                         xl=np.array([-1]*(rooms * slots * days)),                   # Valor mínimo de las variables de decisión
                         xu=np.array([len(lista_clases)-1]*(rooms * slots * days))  # Valor máximo de las variables de decisión                        
                        )

    def _evaluate(self, X, out, *args, **kwargs):

        # Inicializar las matrices que contendrán los valores de la función objetivo y restricciones
        f1 = np.zeros(X.shape[0])
        g = np.zeros((X.shape[0], self.n_constr))

        # Bucle para evaluar cada individuo en la población
        for i in range(X.shape[0]):

            # Extraer el individuo actual y convertirlo a matriz 3D
            ## Aquí habría que meter una función que corrija si el horario no tiene todas las asignaturas repartidas
            individual_3D = deflatten_individual(X[i])

            # individual_3D = corregir_solucion(individual_3D)

            # Evaluar restricciones duras
            g[i, 0] = check_professor_multiple_rooms(individual_3D)
            g[i, 1] = check_package_multiple_events(individual_3D)
            g[i, 2] = check_room_capacity(individual_3D)
            g[i, 3] = verificar_cumplimiento(X[i])

            # Calculando f1: centrado en las preferencias de los estudiantes y profesores
            f1[i] = (alpha1 * cost_student_load(individual_3D, SC_plus) +
                    alpha2 * cost_student_time_preference(individual_3D, student_to_pref) +
                    alpha3 * cost_teacher_load(individual_3D, LC_plus) +
                    alpha4 * cost_teacher_time_preference(individual_3D, teacher_to_pref))
       
        # Asignar los resultados al objeto 'out'
        out["F"] = np.column_stack([f1])
        out["G"] = g

# Instancia del problema
problem_ga = HorarioProblemGA()

# Configuración del algoritmo
algorithm_ga = GA(
    pop_size=pop_size,
    sampling=initial_population,
    crossover=crossover,
    mutation=mutation,
    # selection=selection,
    eliminate_duplicates=True,
)

# Optimizar
res_ga = minimize(problem_ga,
               algorithm_ga,
               termination=termination_ga,
               save_history=True,
               verbose=True,
               seed=seed,
               callback=callback)

# Obtener la mejor solución
mejor_solucion_ga = res_ga.X

## Obtención de mejores parámetros para GA y NSGA2

Ajuste de parámetros para inicializar las pruebas de los algoritmos:

In [ ]:
# Se inicializan las matrices de soluciones
matriz_soluciones_nsga2 = [0] * 31
matriz_fitness_nsga2 = [0] * 31
matriz_soluciones_ga = [0] * 31
matriz_fitness_ga = [0] * 31

# Se define el experimento
experimento = 'exp01'

# Variables globales o atributos de clase (para mutacuión adaptativa)
prev_avg_cv = None
stagnation_counter = 0
original_mutation_rate = 0.005  # Asumimos este % como tasa de mutación inicial para AG y NSGA2

# PARÁMETROS PARA LOS ALGORITMOS GENÉTICOS
pop_size=100

crossover=get_crossover("int_ux")
# crossover=get_crossover("int_hux")
# crossover=get_crossover("int_sbx", prob=0.35)
# crossover=get_crossover("int_k_point", n_points=20)

mutation=get_mutation("int_pm", prob=original_mutation_rate, eta=15)

# selection=get_selection("random")

# Criterio de detención del algorimo

# termination_nsga2=('n_gen', 1000)
# termination_ga=('n_gen', 1000)

termination_nsga2 = MultiObjectiveDefaultTermination(
    x_tol=1e-8,
    cv_tol=1e-6,
    f_tol=0.0025,
    nth_gen=10,
    n_last=200,
    n_max_gen=1000,
    n_max_evals=None
)
termination_ga = SingleObjectiveDefaultTermination(
    x_tol=1e-8,
    cv_tol=1e-6,
    f_tol=1e-6,
    nth_gen=10,
    n_last=200,
    n_max_gen=1000,
    n_max_evals=None
)

Se ejecutan los algoritmos con los parámetros introducidos anteriormente. El único punto que se genera cada vez es la población inicial, ya que se emplea una semilla distinta en cada iteración:

In [ ]:
########## Ejecución de los algoritmos las 31 veces ##########

for i in range(31):
    print(f"Iteración: {i+1}")
    # Creamos una población inicial
    # population = initialize_special_population(population_size=200, top_n=1, seed=i+1)
    population = initialize_mixed_population(population_size=200, hard_ratio=0.7, seed=i+1)
    
    # Crear matriz aplanada de población para pymoo
    population_matrix = flatten_population(population)

    # Preparar la población inicial para introducirla en Pymoo
    initial_population = Population.new("X", population_matrix)

    ######################################################

    # Configuración del algoritmo NSGA2
    algorithm_nsga2 = NSGA2(
        pop_size=pop_size,
        sampling=initial_population,
        crossover=crossover,
        mutation=mutation,
        # selection=selection,
        eliminate_duplicates=True
    )

    # Optimizar con NSGA2
    res_nsga2 = minimize(problem_nsga2,
                        algorithm_nsga2,
                        termination=termination_nsga2,
                        save_history=True,
                        verbose=True,
                        seed=i+1,
                        callback=callback)

    # Obtener la mejor solución
    matriz_soluciones_nsga2[i] = res_nsga2.X
    matriz_fitness_nsga2[i] = res_nsga2.F

    ######################################################

    # Configuración del algoritmo GA
    algorithm_ga = GA(
        pop_size=pop_size,
        sampling=initial_population,
        crossover=crossover,
        mutation=mutation,
        # selection=selection,
        eliminate_duplicates=True,
    )

    # Optimizar con GA
    res_ga = minimize(problem_ga,
                algorithm_ga,
                termination=termination_ga,
                save_history=True,
                verbose=True,
                seed=i+1,
                callback=callback)

    # Obtener la mejor solución
    matriz_soluciones_ga[i] = res_ga.X
    matriz_fitness_ga[i] = res_ga.F

# Se guardan las soluciones
np.save(f'{experimento}_matriz_soluciones_nsga2.npy', matriz_soluciones_nsga2)
np.save(f'{experimento}_matriz_fitness_nsga2.npy', matriz_fitness_nsga2)
np.save(f'{experimento}_matriz_soluciones_ga.npy', matriz_soluciones_ga)
np.save(f'{experimento}_matriz_fitness_ga.npy', matriz_fitness_ga)

Se cargan en un diccionario los datos guardados en la lista `experimentos` para manejarlos después convenientemente:

In [ ]:
# Crear un diccionario para almacenar los datos
datos_experimentos = {}

# Definir los nombres de los archivos base
experimentos = ['exp01', 'exp02', 'exp03', 'exp04', 'exp05',
                'exp06', 'exp07', 'exp08', 'exp09', 'exp10', 'exp11']
tipos = ['nsga2', 'ga']

# Bucle para cargar los datos
for exp in experimentos:
    for tipo in tipos:
        clave_soluciones = f"{exp}_matriz_soluciones_{tipo}"
        clave_fitness = f"{exp}_matriz_fitness_{tipo}"
        archivo_soluciones = f"Resultados pruebas/{exp}_matriz_soluciones_{tipo}.npy"
        archivo_fitness = f"Resultados pruebas/{exp}_matriz_fitness_{tipo}.npy"

        # Cargar los datos y almacenarlos en el diccionario
        datos_experimentos[clave_soluciones] = np.load(archivo_soluciones, allow_pickle=True)
        datos_experimentos[clave_fitness] = np.load(archivo_fitness, allow_pickle=True)

# # Ahora se puede acceder a los datos utilizando las claves del diccionario, por ejemplo:
# exp01_fitness_ga = datos_experimentos['exp01_matriz_fitness_ga']

Esta rutina toma todos los experimentos realizados para el algoritmo GA y calcula sus estadísticos (media, desviación estándar, etc.), para luego generar un dataframe de Pandas y guardarlo en formato Excel:

In [ ]:
# RESULTADOS DE LOS EXPERIMENTOS DE GA

# Inicializamos una lista vacía para almacenar los datos agregados de cada experimento
datos_resumen_ga = []

# Calculamos los valores agregados para cada experimento
for exp in experimentos:
    clave_fitness = f"{exp}_matriz_fitness_ga"
    fitness_data = datos_experimentos[clave_fitness]

    # Aquí asumimos que cada fila en tus datos es una ejecución y cada columna es un individuo.
    # Calculamos las métricas de interés
    fitness_medio = np.mean(fitness_data)  # Promedio de todos los individuos de todas las ejecuciones
    fitness_std = np.std(fitness_data)     # Desviación estándar de todos los individuos de todas las ejecuciones
    fitness_max = np.max(fitness_data)     # Máximo de todos los individuos de todas las ejecuciones
    fitness_min = np.min(fitness_data)     # Mínimo de todos los individuos de todas las ejecuciones

    # Agregamos los datos al resumen
    datos_resumen_ga.append({
        'Experimento': exp,
        'Fitness Medio': fitness_medio,
        'Desviación Estándar': fitness_std,
        'Fitness Máximo': fitness_max,
        'Fitness Mínimo': fitness_min
    })

# Creamos un DataFrame con los datos resumen
df_resumen_ga = pd.DataFrame(datos_resumen_ga)

# Ordenamos por el nombre del experimento si es necesario
df_resumen_ga.sort_values(by='Experimento', inplace=True)

# Mostramos la tabla resumen
print(df_resumen_ga.to_string(index=False))

# Para exportar a Excel
df_resumen_ga.to_excel('fitness_medio_ga.xlsx', index=False)

Este fragmento de código compara la significancia estadística entre dos muestras mediante el test de Wilcoxon. Se emplea para el Algoritmo Genético:

In [ ]:
# Supongamos que queremos trabajar con los datos de 'exp01' y 'nsga2'
clave_exp_1 = 'exp03'
clave_tipo_1 = 'ga'
clave_exp_2 = 'exp04'
clave_tipo_2 = 'ga'

# Construimos las claves del diccionario basadas en lo que necesitas
clave_fitness_1 = f"{clave_exp_1}_matriz_fitness_{clave_tipo_1}"
clave_fitness_2 = f"{clave_exp_2}_matriz_fitness_{clave_tipo_2}"

# Accedemos a los datos directamente desde el diccionario
fitness_1 = datos_experimentos[clave_fitness_1]
fitness_2 = datos_experimentos[clave_fitness_2]

stat, p = wilcoxon(fitness_1, fitness_2)
print(f"Estadístico de Wilcoxon: {stat}, p-valor: {p}")

Por otro lado, para realizar en análisis estadístico del algoritmo NSGA-II, se debe calcular el Hipervolumen. Pero antes hay que calcular el punto de referencia. Para ello obtendremos el peor de los puntos dentro de la matriz de soluciones, para todos los experimentos que se han realizado y se le suma un pequeño épsilon:

In [ ]:
# Cálculo del punto de referencia para Hiper-Volumen

# Inicializamos las variables para almacenar los máximos globales
max_global_f1 = float('-inf')
max_global_f2 = float('-inf')

epsilon = .1  # Definimos un pequeño valor positivo para alejarnos del peor caso

# Iteramos sobre cada experimento para actualizar los máximos globales
for clave_exp in experimentos:
    clave_fitness = f"{clave_exp}_matriz_fitness_nsga2"
    datos = datos_experimentos[clave_fitness]
    
    # Encontramos el máximo para cada función objetivo en este experimento
    max_f1_exp = max(array[:,0].max() for array in datos if array.size > 0)
    max_f2_exp = max(array[:,1].max() for array in datos if array.size > 0)
    
    # Actualizamos los máximos globales si los máximos del experimento actual son mayores
    max_global_f1 = max(max_global_f1, max_f1_exp)
    max_global_f2 = max(max_global_f2, max_f2_exp)

# Definimos el punto de referencia global sumando epsilon para asegurarnos de que sea peor que cualquier solución
punto_referencia_global = (max_global_f1 + epsilon, max_global_f2 + epsilon)  # se puede utilizar en la función de cálculo de hipervolumen

Se define la siguiente función para calcular el hipervolumen:

In [ ]:
# Función que calcula los hyper-volúmenes para cada experimento

def calculo_hypervolumen(clave_exp, punto_referencia=punto_referencia_global):

    # Construimos las claves del diccionario basadas en lo que necesitas
    clave_fitness_1 = f"{clave_exp}_matriz_fitness_nsga2"

    # Accedemos a los datos directamente desde el diccionario
    datos = datos_experimentos[clave_fitness_1]

    # Suponiendo que 'datos' es una lista de arrays de NumPy como antes
    datos_limpios = pd.DataFrame()  # Inicializa un DataFrame vacío para almacenar todos los datos consolidados

    for ejecucion in datos:
        # Convierte a un DataFrame de pandas para facilitar la manipulación
        df = pd.DataFrame(ejecucion, columns=['Objetivo 1', 'Objetivo 2'])
        # Elimina los duplicados
        df = df.drop_duplicates().reset_index(drop=True)
        # (Opcional) Normaliza los datos si es necesario
        # df = (df - df.min()) / (df.max() - df.min())
        # Concatena con el DataFrame de datos_limpios
        datos_limpios = pd.concat([datos_limpios, df], ignore_index=True)

    # Aquí, 'punto_referencia' debe ser un array de NumPy que contiene el punto de referencia

    hv = get_performance_indicator("hv", ref_point=punto_referencia)

    # 'datos_limpios' debe ser un array de NumPy o una lista de listas con las soluciones
    # Si 'datos_limpios' es un DataFrame de pandas, se puede convertir a un array de NumPy así:
    soluciones = datos_limpios.to_numpy()

    # Ahora calcula el hipervolumen con todas tus soluciones
    resultado_hv = hv.calc(soluciones)

    # print("Hipervolumen:", resultado_hv)
    return resultado_hv

Se genera una lista de tuplas con los hipervolúmenes de todos los experimentos realizados:

In [ ]:
# Cálculo de los hyper-volúmenes y almacenamiento en un dataframe
hipervolumenes = []

# Para cada experimento, calcula el hipervolumen y añádelo a la lista
for clave_exp in experimentos:  # Lista de tus claves de experimento
    hv = calculo_hypervolumen(clave_exp)
    hipervolumenes.append((clave_exp, hv))

# Ahora 'hipervolumenes' es una lista de tuplas donde cada tupla contiene la clave del experimento y el hipervolumen calculado.

Ordenamos y guardamos los hipervolúmenes calculados:

In [ ]:
# ORDENAMOS LOS HYPER-VOLUMENES

# Crear un DataFrame directamente de la lista de hipervolumenes
df_hv = pd.DataFrame(hipervolumenes, columns=['Experimento', 'Hipervolumen'])

# Ordenar los valores de hipervolumen de mayor a menor
df_hv_ordenado = df_hv.sort_values(by='Hipervolumen', ascending=False).reset_index(drop=True)

# # Guardar el dataframe en Excel
# df_hv_ordenado.to_excel('hipervolumenes_nsga2.xlsx', index=False)

Se realiza un diagrama de barras para visualizar los hipervolumenes calculados y ordenados:

In [ ]:
# VISUALIZAMOS LOS HIPERVOLUMENES

# Gráfico de barras
df_hv_ordenado.plot(kind='bar', x='Experimento', y='Hipervolumen', legend=False)
plt.ylabel('Hipervolumen')
plt.title('Comparación de Hipervolumen entre Experimentos')
plt.tight_layout()  # Ajusta automáticamente los parámetros de la figura para que quepa todo.
plt.show()

Por último, se realiza un análisis estadístico de los hipervolúmenes calculados:

In [ ]:
# ANÁLISIS ESTADÍSTICO

# Media del hipervolumen
media_hv = df_hv['Hipervolumen'].mean()

# Mediana del hipervolumen
mediana_hv = df_hv['Hipervolumen'].median()

# Desviación estándar del hipervolumen
std_hv = df_hv['Hipervolumen'].std()

# Máximo y mínimo
max_hv = df_hv['Hipervolumen'].max()
min_hv = df_hv['Hipervolumen'].min()

print(f"Media del Hipervolumen: {media_hv}")
print(f"Mediana del Hipervolumen: {mediana_hv}")
print(f"Desviación Estándar del Hipervolumen: {std_hv}")
print(f"Hipervolumen Máximo: {max_hv}")
print(f"Hipervolumen Mínimo: {min_hv}")

## RESULTADOS DE NSGA-II Y GA

Se imprime la mejor solución encontrada por el algoritmo NSGA-II, que está almacenada en la variable `mejor_solucion_nsga2`.

In [ ]:
print(mejor_solucion_nsga2)

Este código recorre la lista de las mejores soluciones obtenidas por el algoritmo NSGA-II, almacenada en la variable `mejor_solucion_nsga2`. Para cada solución:

1. Convierte la solución aplanada en una matriz 3D usando la función `deflatten_individual`.
2. Comprueba las restricciones suaves (`soft constraints`) de la matriz 3D utilizando la función `check_soft_constraints`.
3. Imprime el resultado de verificar las restricciones suaves para cada solución.

In [ ]:
for i in range(len(mejor_solucion_nsga2)):
    horar = deflatten_individual(mejor_solucion_nsga2[i])
    print(check_soft_constraints(horar))

Se redimensiona la matriz `mejor_solucion_ga` para que tenga una fila y `rooms*slots*days` columnas. Luego imprime la matriz redimensionada.

In [ ]:
mejor_solucion_ga = mejor_solucion_ga.reshape(1,rooms*slots*days)
print(mejor_solucion_ga)

Este fragmento de código recorre cada elemento de la matriz `mejor_solucion_ga`. Transforma cada elemento en una representación tridimensional del horario utilizando la función `deflatten_individual`. Luego, verifica las restricciones suaves en este horario tridimensional con la función `check_soft_constraints` e imprime los resultados.

In [ ]:
for i in range(len(mejor_solucion_ga)):
    horar = deflatten_individual(mejor_solucion_ga[i])
    print(check_soft_constraints(horar))

### Reconstruccion de soluciones y guardado en Excel

Este fragmento de código realiza tareas similares para diferentes soluciones: `mejor_solucion_nsga2` y `mejor_solucion_ga`.

1. `reassign_population(mejor_solucion_nsga2/ga)`: convierte los enteros en las matrices de solución en las tuplas de profesores y asignaturas correspondientes.
2. `reconstruct_population(mejor_solucion_profesores)`: toma estas tuplas y las convierte de nuevo en una matriz tridimensional del horario.

Primero se realiza la conversión a tuplas para poder entender qué representan los números (asignaturas y profesores). Luego, estas tuplas se convierten de nuevo en una matriz tridimensional para obtener el horario en un formato más manejable.

In [ ]:
mejor_solucion_profesores = reassign_population(mejor_solucion_nsga2)               # Convertimos los enteros de 'mejor_solucion' en las tuplas correspondientes
mejor_solucion_final_nsga2 = reconstruct_population(mejor_solucion_profesores)      # Volvemos a convertir el horario en una matriz 3D

mejor_solucion_profesores = reassign_population(mejor_solucion_ga)                  # Convertimos los enteros de 'mejor_solucion' en las tuplas correspondientes
mejor_solucion_final_ga = reconstruct_population(mejor_solucion_profesores)         # Volvemos a convertir el horario en una matriz 3D

Estas líneas de código utilizan la función `guarda_resultados` para almacenar las soluciones óptimas finales (`mejor_solucion_final_nsga2` y `mejor_solucion_final_ga`) en archivos. Los nombres de los archivos empiezan con "res_opt_nsga2_" y "res_opt_ga_", respectivamente.

In [ ]:
# guarda_resultados(mejor_solucion_final_nsga2, "res_opt_nsga2_")
# guarda_resultados(mejor_solucion_final_ga, "res_opt_ga_")

### Presentar gráficas de resultados NSGA-II y GA

Se extrae la historia de la optimización realizada por el algoritmo NSGA-II y guarda los valores mínimos de las dos funciones objetivo (`f1` para estudiantes y `f2` para profesores) en cada generación. Posteriormente, grafica estos valores mínimos para cada generación.

Se utiliza la librería matplotlib para mostrar las gráficas.

In [ ]:
# Extraer la historia
history = res_nsga2.history

# Listas para guardar los valores mínimos de f1 y f2 en cada generación
f1_values = []
f2_values = []

# Extraer los valores mínimos de cada función objetivo en cada generación
for gen in history:
    f1_values.append(np.min(gen.pop.get("F")[:, 0]))
    f2_values.append(np.min(gen.pop.get("F")[:, 1]))

# Graficar
plt.figure()
plt.plot(f1_values, label="f1 (Estudiantes)")
plt.plot(f2_values, label="f2 (Profesores)")
plt.xlabel("Generación")
plt.ylabel("Valor de la función objetivo")
plt.legend()
plt.show()

Aquí se hace algo similar pero para un algoritmo genético con un solo objetivo (GA). En este caso, solo se guarda el valor mínimo de la función objetivo `f1`, que considera tanto a estudiantes como a profesores, en cada generación. Luego, grafica estos valores mínimos a lo largo de las generaciones.

También se utiliza la librería matplotlib para mostrar las gráficas.

In [ ]:
# Extraer la historia
history = res_ga.history

# Listas para guardar los valores mínimos de f1 y f2 en cada generación
f1_values = []

# Extraer los valores mínimos de cada función objetivo en cada generación
for gen in history:
    f1_values.append(np.min(gen.pop.get("F")[:, 0]))

# Graficar
plt.figure()
plt.plot(f1_values, label="f1 (Estudiantes + Profesores)")
plt.xlabel("Generación")
plt.ylabel("Valor de la función objetivo")
plt.legend()
plt.show() 

### Guardar los resultados de NSGA-II y GA en Excel

Se guardan los resultados de los algoritmos NSGA-II y GA en archivos Excel. Los archivos se nombran "resultados_nsga2.xlsx" y "resultados_ga.xlsx", respectivamente.

In [ ]:
# # Permite guardar el objeto resultado de los algoritmos en un Excel
# guardar_resultados_pymoo_en_excel(res_nsga2, "resultados_nsga2.xlsx")
# guardar_resultados_pymoo_en_excel(res_ga, "resultados_ga.xlsx")